In [31]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F # importante para usar as funções do spark

spark = (
    SparkSession
    .builder
        .master("local[*]")
    .appName('spark_dataframe_api')
    .getOrCreate()
)

# configurando as opções e lendo o csv
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true') # tenta inferir o schema
    .option('enconding', 'ISO-8859-1') # as vezes está especificado no arquivo
    .csv('./precos-gasolina-etanol-10.csv') # acesso relativo ao arquivo
)

df.printSchema()

24/08/16 13:38:06 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [35]:
(
    df
    .where( # semelhante ao where do SQL
        F.col('Valor de Compra').isNotNull()
    )
    .show()
)

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



- analisar a variação do preço do combustível em um estado

In [34]:
df_precos = (
    df
    .select('Estado - Sigla', 'Produto', 'Valor de Venda', 'Unidade de Medida')
    .withColumn(
        "Valor de Venda", # nome da coluna para criar ou manipular
        F.regexp_replace(F.col('Valor de Venda'), ',', '.')
        .cast('float') # atribuição
    )
)

df_precos.show()

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AL|          GASOLINA|          6.08|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.08|       R$ / litro|
|            AL|            ETANOL|          4.78|       R$ / litro|
|            AL|          GASOLINA|          5.79|       R$ / litro|
|            AL|            ETANOL|          4.29|       R$ / litro|
|            AL|          GASOLINA|          5.89|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.09|       R$ / litro|
|            AL|            ETANOL|          4.49|       R$ / litro|
|            AL|          GASOLINA|          5.89|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.09|       R$ / litro|
|            AL|            ETANOL|          4.99|       R$ / litro|
|            AL|          GASOLINA

In [37]:
(
    df_precos
    .groupBy(
        F.col('Estado - Sigla'),
        F.col('Produto'),
        F.col('Unidade de Medida')
    )
    .agg(
        F.min(F.col('Valor de Venda')).alias('menor_valor'),
        F.max(F.col('Valor de Venda')).alias('maior_valor')
    )
    .withColumn(
        'diferenca',
        F.col('maior_valor') - F.col('menor_valor')
    )
    .orderBy('diferenca', ascending=False)
    .show()
)

+--------------+------------------+-----------------+-----------+-----------+---------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|diferenca|
+--------------+------------------+-----------------+-----------+-----------+---------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       4.79|       8.69|3.8999996|
|            SP|            ETANOL|       R$ / litro|       2.78|       6.19|     3.41|
|            SP|          GASOLINA|       R$ / litro|       4.59|       7.59|      3.0|
|            PA|            ETANOL|       R$ / litro|       3.85|        6.6|     2.75|
|            RS|            ETANOL|       R$ / litro|       3.88|       6.29|2.4099998|
|            BA|          GASOLINA|       R$ / litro|       4.69|       6.98|     2.29|
|            SC|            ETANOL|       R$ / litro|       3.84|       5.89|     2.05|
|            AL|            ETANOL|       R$ / litro|       3.79|        5.8|2.0100002|
|            PE|            ETAN